In [ ]:
import pickle
import json
import random
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open("datasets/smallerdata.json", 'r') as f:
    tweets = json.load(f)
random.shuffle(tweets)
train = tweets[:int(round(4*len(tweets)/5))]
test = tweets[int(round(4*len(tweets)/5)):len(tweets)]
xtrain = []
ytrain = []
xtest = []
ytest = []

for tweet in train:
    xtrain.append(tweet['content'])
    ytrain.append(tweet['label'])
    
for tweet in test:
    xtest.append(tweet['content'])
    ytest.append(tweet['label'])

    
print(xtrain[:5])
print(ytrain[:5])
print(xtest[:5])
print(ytest[:5])

In [ ]:
# preprocessklearn.py file (you are actually supposed to import this but for demonstration i have just put this in)
# you can do this by putting the line "from preprocesssklearn import *"
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
def removespchar(text):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

def removestopwords(text, islowercase=True):
    tokenizer=ToktokTokenizer()
    stopwordList=nltk.corpus.stopwords.words('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if islowercase:
        filteredTokens = [token for token in tokens if token not in stopwordList]
    else:
        filteredTokens = [token for token in tokens if token.lower() not in stopwordList]
    filteredText = ' '.join(filteredTokens)    
    return filteredText

In [ ]:
for i in range(len(xtrain)):
    xtrain[i] = removespchar(xtrain[i])
    xtrain[i] = stemmer(xtrain[i])
    xtrain[i] = removestopwords(xtrain[i])

for i in range(len(xtest)):
    xtest[i] = removespchar(xtest[i])
    xtest[i] = stemmer(xtest[i])
    xtest[i] = removestopwords(xtest[i])
    
print(xtrain[:5])
print(xtest[:5])

In [ ]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
xtrain=tv.fit_transform(xtrain)
xtest=tv.transform(xtest)

In [ ]:
print(xtrain[0])

In [ ]:
%matplotlib inline

In [ ]:
svm=LinearSVC()
svm.fit(xtrain,ytrain)
svmpred = svm.predict(xtest)
print(svmpred[:5])
print(ytest[:5])
print("Support Vector Machine Accuracy Score -> ",accuracy_score(svmpred, ytest)*100)

In [ ]:
array = confusion_matrix(ytest,svmpred,labels=[1,0])
df_cm = pd.DataFrame(array, range(2), range(2))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})
plt.show()

In [ ]:
nb=naive_bayes.MultinomialNB()
nb.fit(xtrain,ytrain)
nbpred = nb.predict(xtest)
print(nbpred[:5])
print(ytest[:5])
print("Naive Bayes Accuracy Score -> ",accuracy_score(nbpred, ytest)*100)

In [ ]:
array = confusion_matrix(ytest,nbpred,labels=[1,0])
df_cm = pd.DataFrame(array, range(2), range(2))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})
plt.show()

In [ ]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr.fit(xtrain,ytrain)
lrpred = lr.predict(xtest)
print(lrpred[:5])
print(ytest[:5])
print("Logistic Regression Accuracy Score -> ",accuracy_score(lrpred, ytest)*100)

In [ ]:
array = confusion_matrix(ytest,lrpred,labels=[1,0])
df_cm = pd.DataFrame(array, range(2), range(2))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})
plt.show()

In [ ]:
import pickle
from preprocesssklearn import *
pickle.dump(cv, open("savedModel/sklearnpynb/BoW.sav","wb"))
pickle.dump(nb, open("savedModel/sklearnpynb/nb.sav","wb"))
pickle.dump(svm, open("savedModel/sklearnpynb/svm.sav","wb"))
pickle.dump(lr, open("savedModel/sklearnpynb/lr.sav","wb"))

In [ ]:
n = int(input("no of sentences: "))
sentences = [str(input("enter sentence:")) for _ in range(n)]

In [ ]:
for i in range(len(sentences)):
    sentences[i] = removespchar(sentences[i])
    sentences[i] = stemmer(sentences[i])
    sentences[i] = removestopwords(sentences[i])

In [ ]:
cv = pickle.load(open("savedModel/sklearnpynb/BoW.sav","rb"))
nb = pickle.load(open("savedModel/sklearnpynb/nb.sav","rb"))
svm = pickle.load(open("savedModel/sklearnpynb/svm.sav","rb"))
lr = pickle.load(open("savedModel/sklearnpynb/lr.sav","rb"))

In [ ]:
sentences = cv.transform(sentences)
print(svm.predict(sentences))
print(nb.predict(sentences))
print(lr.predict(sentences))